In [1]:
import polars as pl
import os

In [2]:
pl.Config(fmt_str_lengths=1000)

In [3]:
files = os.listdir('data/domain')

In [4]:
df_emp = pl.read_parquet(f'data/domain/{files[2]}')

In [7]:
df_emp.head(6)

cnpj_completo,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,pais,data_inicio_atividade,cnae_principal,cnae_secundario,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,fax,email,situacao_especial,data_situacao_especial,logradouro_completo
str,i8,str,i8,date,i16,str,i32,date,i64,list[str],str,str,str,str,cat,i32,str,str,str,str,str,str,str,cat,date,str
"""27875213000113""",1,null,2,2024-01-18,0,"""""",null,2017-06-01,9001906,"[""""]","""120""","""BLOCO I;APT 03""","""BAETA NEVES""","""09760410""","""SP""",7075,"""11""","""23747104""",null,null,"""""","""""",null,"""""",null,"""RUA OSEAS DE PAULA CAMPOS"""
"""26244670000570""",2,"""ROTA SERVICE ILHA DO GOVERNADOR""",8,2023-02-24,1,"""""",null,2019-11-13,4541203,"[""4520007,4541204,4541206,4781400""]","""00529""",""" LOJ A""","""JARDIM GUANABARA""","""21931383""","""RJ""",6001,"""21""","""40404033""",null,null,"""""","""""",null,"""""",null,"""ESTRADA DO GALEAO"""
"""35509654000120""",1,null,2,2019-11-13,0,"""""",null,2019-11-13,5611201,"[""5611203""]","""510""","""LETRA A""","""PEDRINHA""","""12508010""","""SP""",6469,"""12""","""31276182""",null,null,"""""","""""","""LCBRJ@TERRA.COM.BR""","""""",null,"""ESTRADA PRESIDENTE TANCREDO NEVES"""
"""35510044000146""",1,null,2,2023-01-31,0,"""""",null,2019-11-13,4772500,"[""""]","""89""",null,"""JARDIM PLANALTO""","""59155256""","""RN""",1779,"""84""","""98566127""",null,null,"""""","""""","""IRLAFRANCELE@HOTMAIL.COM""","""""",null,"""RUA CONCORDE"""
"""35510441000118""",1,"""BAR DO JONAS""",8,2019-11-13,1,"""""",null,2019-11-13,5611204,"[""4729601,4723700,5611201,5611203""]","""1100""","""LOJA""","""MARIA ORTIZ""","""29070350""","""ES""",5705,"""27""","""99622587""",null,null,"""""","""""","""MARCELO.NET77@HOTMAIL.COM""","""""",null,"""AVENIDA JERONIMO VERVLOET"""
"""35510848000145""",1,null,2,2019-11-13,0,"""""",null,2019-11-13,1052000,"[""5612100""]","""3351""","""CASA;CASA;CASA;CASA""","""CONSELHEIRO PAULINO""","""28635010""","""RJ""",5867,"""22""","""98231774""",null,null,"""""","""""","""JAIMEJUNIOR610@GMAIL.COM""","""""",null,"""AVENIDA NOSSA SENHORA DO AMPARO"""


In [ ]:
#Cleaning ddd's fields

In [22]:
df_emp = df_emp.with_columns(
    pl.when(~pl.col('ddd_1').str.contains("-?\d+(?:\.\d+)?")).then(pl.lit(None)).otherwise(pl.col('ddd_1')).alias('ddd1_alias').str.strip_chars().cast(pl.Int32),
    pl.when(~pl.col('ddd_2').str.contains("-?\d+(?:\.\d+)?")).then(pl.lit(None)).otherwise(pl.col('ddd_2')).alias('ddd2_alias').str.strip_chars().cast(pl.Int32)
).drop(['ddd_1','ddd_2'])

In [158]:
### change to matriz / filial

In [24]:
df_emp = df_emp.with_columns(
    pl.col('identificador_matriz').replace([1,2],['matriz','filial']).cast(pl.Categorical)
)

##### Support table - motivo

In [25]:
support_motivo_path = os.listdir('data/raw/support/motivos/')[0]

In [26]:
motivo_column = [
    'situacao_cadastral',
    'descricao_motivo'
]

In [54]:
dtypes_motivo = {
    'situacao_cadastral':pl.Int8,
    'descricao_motivo':pl.Categorical
}

In [56]:
df_mot = pl.read_csv(f'data/raw/support/motivos/{support_motivo_path}', separator = ';', new_columns = motivo_column,dtypes=dtypes_motivo)

##### Support table - municipios

In [29]:
support_municipio_path = os.listdir('data/raw/support/municipios/')[0]

In [30]:
municipio_column = [
    'municipio',
    'descricao_municipio'
]

In [31]:
dtypes_municipio = {
    'municipio':pl.Int32
}

In [32]:
df_mun = pl.read_csv(f'data/raw/support/municipios/{support_municipio_path}',separator = ';',new_columns = municipio_column, dtypes=dtypes_municipio)

##### Support table - pais

In [33]:
support_pais_path = os.listdir('data/raw/support/paises/')[0]

In [34]:
pais_column = [
    'pais',
    'descricao_pais'
]

In [35]:
dtypes_pais = {
    'pais':pl.Int32
}

In [36]:
df_paises = pl.read_csv(f'data/raw/support/paises/{support_pais_path}',separator = ';',encoding='latin1', new_columns = pais_column, dtypes=dtypes_pais)

##### Joining data

In [58]:
#sit. cadastral
df_emp = df_emp.join(df_mot, on="situacao_cadastral",how='left')

In [38]:
#municipio
df_emp = df_emp.join(df_mun, on="municipio",how='left')

In [39]:
#paises
df_emp = df_emp.join(df_paises, on="pais",how='left')

In [59]:
select_columns = [
    'cnpj_completo',
    'nome_fantasia',
    'identificador_matriz',
    'data_inicio_atividade',
    'descricao_pais',
    'uf',
    'descricao_municipio',
    'cep',
    'logradouro_completo',
    'bairro',
    'numero',
    'complemento',
    'ddd1_alias',
    'telefone_1',
    'ddd2_alias',
    'telefone_2',
    'email',
    'cnae_principal',
    'descricao_motivo'
]

In [60]:
df_new = df_emp.select(select_columns)

In [61]:
df_new.head(2)

cnpj_completo,nome_fantasia,identificador_matriz,data_inicio_atividade,descricao_pais,uf,descricao_municipio,cep,logradouro_completo,bairro,numero,complemento,ddd1_alias,telefone_1,ddd2_alias,telefone_2,email,cnae_principal,descricao_motivo
str,str,cat,date,str,cat,str,str,str,str,str,str,i32,str,i32,str,str,i64,str
"""27875213000113""",null,"""matriz""",2017-06-01,null,"""SP""","""SAO BERNARDO DO CAMPO""","""09760410""","""RUA OSEAS DE PAULA CAMPOS""","""BAETA NEVES""","""120""","""BLOCO I;APT 03""",11,"""23747104""",null,null,null,9001906,"""INCORPORACAO"""
"""26244670000570""","""ROTA SERVICE ILHA DO GOVERNADOR""","""filial""",2019-11-13,null,"""RJ""","""RIO DE JANEIRO""","""21931383""","""ESTRADA DO GALEAO""","""JARDIM GUANABARA""","""00529""",""" LOJ A""",21,"""40404033""",null,null,null,4541203,"""TRANSPASSE"""
